In [1]:
import pandas as pd
import numpy as np

In [2]:
udaje_atleti = pd.read_csv('athlete_events.csv')
regiony = pd.read_csv('noc_regions.csv')

In [3]:
data = pd.merge(udaje_atleti, regiony, on='NOC', how='left')

In [4]:
data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992.0,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012.0,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920.0,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900.0,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988.0,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


Potrebujeme vymazat riadky s hodnotami null v stlpcoch "age", "height", "weight", "region". 

In [5]:
data = data.dropna(axis=0, subset=['Age', 'Height', 'Weight', 'region'])

Rovnako vymazeme stlpec notes, ktory je pre nepotrebny. 
Nakoniec nahradime null hodnoty v stlpci "Medals" na hodnotu "None"

In [6]:
data = data.drop(['notes', 'ID', 'Name', 'Team', 'NOC', 'Games', 'Season', 'City'], axis=1)

In [7]:
data.head()

,Sex,Age,Height,Weight,Year,Sport,Event,Medal,region
0,M,24.0,180.0,80.0,1992.0,Basketball,Basketball Men's Basketball,NaN,China
1,M,23.0,170.0,60.0,2012.0,Judo,Judo Men's Extra-Lightweight,NaN,China
4,F,21.0,185.0,82.0,1988.0,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands
5,F,21.0,185.0,82.0,1988.0,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,Netherlands
6,F,25.0,185.0,82.0,1992.0,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands


Este si pre istotu mozeme skontrolovat ci uz sa v nasom datasete nenachadzaju ziadne null hodnoty okrem stlpca "Medal"

In [8]:
pd.isnull(data).sum()

Sex            0
Age            0
Height         0
Weight         0
Year           0
Sport          0
Event          0
Medal     107331
region         0
dtype: int64

In [9]:
data['haveMedal'] = np.where(data['Medal'].isnull(), 0, 1)

In [10]:
data.head()

,Sex,Age,Height,Weight,Year,Sport,Event,Medal,region,haveMedal
0,M,24.0,180.0,80.0,1992.0,Basketball,Basketball Men's Basketball,NaN,China,0
1,M,23.0,170.0,60.0,2012.0,Judo,Judo Men's Extra-Lightweight,NaN,China,0
4,F,21.0,185.0,82.0,1988.0,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,0
5,F,21.0,185.0,82.0,1988.0,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,Netherlands,0
6,F,25.0,185.0,82.0,1992.0,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,0


In [13]:
X = data.iloc[:,1:5]
y = data.iloc[:,9]

In [14]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [58]:
model = Sequential()
model.add(Dense(16, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [59]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.fit(X, y, epochs=5, batch_size=1000)
scores=model.evaluate(X,y)

Epoch 1/5
125357/125357 [==============================] - 1s 5us/step - loss: 0.9328 - accuracy: 0.7743
Epoch 2/5
125357/125357 [==============================] - 0s 3us/step - loss: 0.5547 - accuracy: 0.8165
Epoch 3/5
125357/125357 [==============================] - 0s 3us/step - loss: 0.5892 - accuracy: 0.8114
Epoch 4/5
125357/125357 [==============================] - 0s 3us/step - loss: 0.4340 - accuracy: 0.8488
Epoch 5/5
125357/125357 [==============================] - 4s 28us/step


In [61]:
print("\nPresnost siete: %.2f%%"%(scores[1]*100))


Presnost siete: 85.01%
